In [2]:
import warnings

# Hide all warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install seaborn

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [4]:
#   Load the three raw datasets  
try:
    df_employee = pd.read_csv('data/employee_data.csv')
    df_insurance = pd.read_csv('data/insurance_data.csv')
    df_vendor = pd.read_csv('data/vendor_data.csv')
    print("All three raw datasets loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading raw datasets: {e}")
    print("Please ensure 'employee_data.csv', 'insurance_data.csv', and 'vendor_data.csv' are in a 'data' subdirectory.")

All three raw datasets loaded successfully.


 Initial Cleaning and Merging

In [5]:
#   Perform initial cleaning and merging from EDA phase  

# 1. Convert Date Columns
date_cols_insurance = ['TXN_DATE_TIME', 'POLICY_EFF_DT', 'LOSS_DT', 'REPORT_DT']
for col in date_cols_insurance:
    df_insurance[col] = pd.to_datetime(df_insurance[col], errors='coerce')

df_employee['DATE_OF_JOINING'] = pd.to_datetime(df_employee['DATE_OF_JOINING'], errors='coerce')

# 2. Merge DataFrames
df_merged = pd.merge(df_insurance, df_employee, on='AGENT_ID', how='left', suffixes=('', '_emp'))
df_merged = pd.merge(df_merged, df_vendor, on='VENDOR_ID', how='left', suffixes=('', '_vend'))

# 3. Create Target Variable (FRAUD_FLAG)
status_to_fraud_map = {'A': 0, 'D': 1} # A=Approved (Legit), D=Denied (Fraud)
df_merged['FRAUD_FLAG'] = df_merged['CLAIM_STATUS'].map(status_to_fraud_map)

# 4. Save the merged file (this resolves the error)
df_merged.to_csv('data/merged_insurance_data.csv', index=False)

print("Initial cleaning and merging complete.")
print("Merged DataFrame shape:", df_merged.shape)
print("'data/merged_insurance_data.csv' has been created.")

Initial cleaning and merging complete.
Merged DataFrame shape: (10000, 54)
'data/merged_insurance_data.csv' has been created.


Load the Merged Data

In [6]:
#   Load the newly created merged dataset  
try:
    df_merged = pd.read_csv('data/merged_insurance_data.csv')
    # Convert date columns back to datetime objects as they are read as strings from CSV
    date_cols = ['TXN_DATE_TIME', 'POLICY_EFF_DT', 'LOSS_DT', 'REPORT_DT', 'DATE_OF_JOINING']
    for col in date_cols:
        df_merged[col] = pd.to_datetime(df_merged[col])
    print("\n'merged_insurance_data.csv' loaded successfully for feature engineering.")
    print("Shape:", df_merged.shape)
except FileNotFoundError as e:
    print(f"Error: {e}")


'merged_insurance_data.csv' loaded successfully for feature engineering.
Shape: (10000, 54)


Create New Features

In [7]:
#   5a. Time-Based Features  
df_merged['DAYS_POLICY_TO_LOSS'] = (df_merged['LOSS_DT'] - df_merged['POLICY_EFF_DT']).dt.days
df_merged['DAYS_LOSS_TO_REPORT'] = (df_merged['REPORT_DT'] - df_merged['LOSS_DT']).dt.days
df_merged['AGENT_TENURE_AT_REPORT'] = (df_merged['REPORT_DT'] - df_merged['DATE_OF_JOINING']).dt.days
df_merged['TXN_MONTH'] = df_merged['TXN_DATE_TIME'].dt.month
df_merged['TXN_DAY_OF_WEEK'] = df_merged['TXN_DATE_TIME'].dt.dayofweek

#   5b. Interaction and Ratio Features  
df_merged['CLAIM_PREMIUM_RATIO'] = df_merged['CLAIM_AMOUNT'] / (df_merged['PREMIUM_AMOUNT'] + 1e-6)
df_merged['AGE_AT_POLICY_START'] = df_merged['AGE'] - (df_merged['TXN_DATE_TIME'].dt.year - df_merged['POLICY_EFF_DT'].dt.year)

print("\nNew features created successfully.")
print(df_merged[['DAYS_POLICY_TO_LOSS', 'CLAIM_PREMIUM_RATIO', 'TXN_MONTH']].head())


New features created successfully.
   DAYS_POLICY_TO_LOSS  CLAIM_PREMIUM_RATIO  TXN_MONTH
0                 1789            57.277413          6
1                  753           183.473289          6
2                  231            82.676163          6
3                 1262            92.555099          6
4                 3065            33.886818          6


In [8]:
# Drop PII, redundant, and original date columns that are no longer needed for modeling.
#   6. Drop PII and Redundant Columns  
pii_cols = [
    'TRANSACTION_ID', 'CUSTOMER_ID', 'POLICY_NUMBER', 'CUSTOMER_NAME',
    'ADDRESS_LINE1', 'ADDRESS_LINE2', 'CITY', 'POSTAL_CODE', 'SSN',
    'ROUTING_NUMBER', 'ACCT_NUMBER', 'AGENT_NAME', 'DATE_OF_JOINING',
    'ADDRESS_LINE1_emp', 'ADDRESS_LINE2_emp', 'CITY_emp', 'POSTAL_CODE_emp',
    'EMP_ROUTING_NUMBER', 'EMP_ACCT_NUMBER', 'VENDOR_NAME',
    'ADDRESS_LINE1_vend', 'ADDRESS_LINE2_vend', 'CITY_vend', 'POSTAL_CODE_vend',
    'CLAIM_STATUS' # Redundant
]
date_cols_to_drop = ['TXN_DATE_TIME', 'POLICY_EFF_DT', 'LOSS_DT', 'REPORT_DT']
cols_to_drop = pii_cols + date_cols_to_drop

df_processed = df_merged.drop(columns=cols_to_drop, errors='ignore')

print(f"\nDropped {len(cols_to_drop)} unnecessary columns.")
print("Shape of DataFrame for modeling:", df_processed.shape)


Dropped 29 unnecessary columns.
Shape of DataFrame for modeling: (10000, 32)


In [9]:
#   7. Handle Missing Values  
for col in df_processed.select_dtypes(include=np.number).columns:
    if df_processed[col].isnull().any():
        median_val = df_processed[col].median()
        df_processed[col].fillna(median_val, inplace=True)

for col in df_processed.select_dtypes(include=['object', 'category']).columns:
    if df_processed[col].isnull().any():
        mode_val = df_processed[col].mode()[0]
        df_processed[col].fillna(mode_val, inplace=True)

print("\nMissing values handled.")
print("Remaining missing values:", df_processed.isnull().sum().sum())


Missing values handled.
Remaining missing values: 0


Preprocessing for Modeling

In [10]:
# Data Splitting
# Split the data into training and testing sets before applying scaling and encoding to prevent data leakage.

#  8. Train-Test Split 
X = df_processed.drop('FRAUD_FLAG', axis=1)
y = df_processed['FRAUD_FLAG']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\nData split into training and testing sets.")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


Data split into training and testing sets.
X_train shape: (8000, 31)
X_test shape: (2000, 31)


**Preprocessing with Pipelines**

Define pipelines to scale numerical data and one-hot encode categorical data.


In [11]:
## 9. Define and Apply Preprocessing
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=np.number).columns

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='passthrough'
)

# Fit on training data and transform both sets
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Get feature names for the new DataFrame columns
ohe_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
new_column_names = list(numerical_cols) + list(ohe_feature_names)

# Convert back to DataFrames
X_train_processed_df = pd.DataFrame(X_train_processed, columns=new_column_names, index=X_train.index)
X_test_processed_df = pd.DataFrame(X_test_processed, columns=new_column_names, index=X_test.index)

print("\nPreprocessing complete.")
print("Shape of processed training features:", X_train_processed_df.shape)


Preprocessing complete.
Shape of processed training features: (8000, 2115)


In [12]:
#   10. Save Data for Modeling  
train_data = pd.concat([X_train_processed_df, y_train], axis=1)
test_data = pd.concat([X_test_processed_df, y_test], axis=1)

train_data.to_csv('data/train_processed.csv', index=False)
test_data.to_csv('data/test_processed.csv', index=False)

print("\nProcessed data saved and ready for modeling in notebook 03.")


Processed data saved and ready for modeling in notebook 03.
